urgh, I'm really bad at these ones

I guess the very first step is to get the data and find the starting point

In [ ]:
from aocd import get_data, submit

current_day is only available in December (EST)


In [ ]:
data = get_data(year = 2025, day = 7)

In [ ]:
example = """.......S.......
.......|.......
......|^|......
......|.|......
.....|^|^|.....
.....|.|.|.....
....|^|^|^|....
....|.|.|.|....
...|^|^|||^|...
...|.|.|||.|...
..|^|^|||^|^|..
..|.|.|||.|.|..
.|^|||^||.||^|.
.|.|||.||.||.|.
|^|^|^|^|^|||^|
|.|.|.|.|.|||.|"""

In [ ]:
data[:100]

'......................................................................S.............................'

In [ ]:
def parse_data(data):
    data_lines = data.splitlines() 
    data_ls = [[s for s in row] for row in data_lines]

    return data_ls

In [ ]:
data_ls = parse_data(data)
example_ls = parse_data(example)

In [ ]:
def find_s(data_ls):
    n_rows = len(data_ls)
    n_cols = len(data_ls[0])
    
    for r in range(n_rows):
        for c in range(n_cols):
            if data_ls[r][c] == "S":
                return r, c

In [ ]:
find_s(example_ls)

(0, 7)

okay.. now I guess I need to iterate in a continuous loop, stopping when it hits a dash before moving on. this will also need to support splitting into multiple iterators

before I proceed, is that the right approach?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Yes, that's a good approach! You'll need to track multiple beams as they travel downward and split when they hit `^` symbols.

A few things to consider for your implementation:

1. **State tracking**: Each beam needs to know its current position (row, col)
2. **Stopping condition**: A beam stops when it hits a `^` splitter or exits the manifold
3. **Splitting**: When a beam hits `^`, it stops and creates two new beams going left and right from that position
4. **Multiple active beams**: You'll need a way to process all active beams (like a queue or list)

One important question: when a beam splits and creates left/right beams, do those new beams continue moving left/right horizontally, or do they also move downward? Looking at the example diagrams, what do you think?

In [ ]:
def count_splits(data_ls):
    start_r, start_c = find_s(data_ls)
    
    # Each beam is (row, col)
    beams = [(start_r, start_c)]
    splits = 0
    
    while beams:
        new_beams = []
        
        for r, c in beams:
            # Move down one step
            r += 1
            
            # Check if out of bounds
            if r >= len(data_ls):
                continue
                
            # Check what we hit
            if data_ls[r][c] == '^':
                splits += 1
                # Add left and right beams
                new_beams.append((r, c - 1))
                new_beams.append((r, c + 1))
            else:
                # Continue this beam downward
                new_beams.append((r, c))
        
        beams = new_beams
    
    return splits

In [ ]:
count_splits(example_ls)

39

hmm, this is overcounting

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good catch! Think about what happens in the example when two adjacent splitters each send a beam toward each other — they both target the same middle position. 

Look at this part of the problem description:

> "the two splitters create a total of only three tachyon beams, since they are both dumping tachyons into the same place between them"

What does that tell you about how `beams` should handle duplicates?

In [ ]:
def count_splits(data_ls):
    start_r, start_c = find_s(data_ls)
    
    # Each beam is (row, col)
    beams = [(start_r, start_c)]
    splits = 0
    
    while beams:
        new_beams = []
        
        for r, c in beams:
            # Move down one step
            r += 1
            
            # Check if out of bounds
            if r >= len(data_ls):
                continue
                
            # Check what we hit
            if data_ls[r][c] == '^':
                beam_l = (r, c - 1)
                beam_r = (r, c + 1)
                
                beam_l_exist = beam_l in new_beams
                beam_r_exist = beam_r in new_beams

                if beam_l_exist & beam_r_exist:
                    continue
                elif beam_l_exist:
                    new_beams.append(beam_l)
                elif beam_r_exist:
                    new_beams.append(beam_r)
                else:
                    splits += 1
                    new_beams.append(beam_l)
                    new_beams.append(beam_r)
                                            
            else:
                # Continue this beam downward
                new_beams.append((r, c))
        
        beams = new_beams
    
    return splits

In [ ]:
splits = count_splits(example_ls)
splits

7

ffs whats wrong now?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

You're counting a split only when *neither* new beam already exists. But a split happens every time a beam hits a `^`, regardless of whether the resulting beams merge with others.

Think about it: if a beam hits a splitter, that's a split. The merging of beams is a separate concern from counting splits.

Try separating these two tasks:
1. Count a split whenever *any* beam hits a `^`
2. Deduplicate beams separately (hint: a `set` is perfect for this)

In [ ]:
def count_splits(data_ls):
    start_r, start_c = find_s(data_ls)
    
    # Each beam is (row, col)
    beams = [(start_r, start_c)]
    splits = 0
    
    while beams:
        new_beams = []
        
        for r, c in beams:
            # Move down one step
            r += 1
            
            # Check if out of bounds
            if r >= len(data_ls):
                continue
                
            # Check what we hit
            if data_ls[r][c] == '^':
                splits += 1
                # Add left and right beams
                new_beams.append((r, c - 1))
                new_beams.append((r, c + 1))
            else:
                # Continue this beam downward
                new_beams.append((r, c))

        beams = list(set(new_beams))
    
    return splits

In [ ]:
count_splits(example_ls)

21

In [ ]:
pt1_answer = count_splits(data_ls)
pt1_answer

I think this would work but takes far too long to run... what might be some solutions?



##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

The main bottleneck is likely that `list` operations like `in` are slow — checking membership in a list is O(n), which adds up fast when you have lots of beams.

Try using a `set` for `new_beams` from the start, rather than converting at the end. That way `add()` handles deduplication automatically with O(1) lookups.

Does that make sense to try?

In [ ]:
def count_splits(data_ls):
    start_r, start_c = find_s(data_ls)
    
    # Each beam is (row, col)
    beams = set([(start_r, start_c)])
    splits = 0
    
    while beams:
        new_beams = set()
        
        for r, c in beams:
            # Move down one step
            r += 1
            
            # Check if out of bounds
            if r >= len(data_ls):
                continue
                
            # Check what we hit
            if data_ls[r][c] == '^':
                splits += 1
                # Add left and right beams
                new_beams.add((r, c - 1))
                new_beams.add((r, c + 1))
            else:
                # Continue this beam downward
                new_beams.add((r, c))

        beams = new_beams
    
    return splits

In [ ]:
count_splits(example_ls)

21

In [ ]:
pt1_answer = count_splits(data_ls)
pt1_answer

1507

In [ ]:
submit(pt1_answer, part = "a", year = 2025, day = 7)

That's the right answer!  You are one gold star closer to decorating the North Pole. [Continue to Part Two]


hmm, so I think this might be as simple as adjust to make multiple directional movements at each iter

In [ ]:
movements = [
    [(

In [ ]:
def pt2(data_ls):
    start_r, start_c = find_s(data_ls)
    
    # Each beam is (row, col)
    beams = set([(start_r, start_c)])
    splits = 0
    
    while beams:
        new_beams = set()
        
        for r, c in beams:
            # Move down one step
            r += 1
            
            # Check if out of bounds
            if r >= len(data_ls):
                continue
                
            # Check what we hit
            if data_ls[r][c] == '^':
                splits += 1
                # Add left and right beams
                new_beams.add((r, c - 1))
                new_beams.add((r, c + 1))
            else:
                # Continue this beam downward
                new_beams.add((r, c))

        beams = new_beams
    
    return splits

ffs, I don't get this. can you help

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

No worries — this one's a bit of a mind-bender!

The key difference is:

- **Part 1**: Count splits. Beams at the same position merge into one.
- **Part 2**: Count timelines. Even if two particles end up at the same position, they're *different timelines* if they took different paths to get there.

So instead of just tracking *where* beams are (a set of positions), you need to track *how many timelines* are at each position.

What data structure could you use to track "position → number of timelines at that position"?